In [1]:

import pandas as pd

YEARS = range(2021, 2026)

batters = []

for year in YEARS:
    df = pd.read_csv(f"BR_KIWOOM_batting_{year}.csv")
    df["season"] = year
    batters.append(df)

bat_all = pd.concat(batters, ignore_index=True)

# 합계 행 제거 (Rk 없는 행)
bat_all = bat_all[bat_all["Rk"].notna()].copy()

# 이름 정리 (* 제거)
bat_all["Name"] = bat_all["Name"].astype(str).str.replace("*", "", regex=False)

# 숫자형 컬럼 정리 (중요)
num_cols_bat = [
    "Age","G","PA","AB","R","H","2B","3B","HR","RBI",
    "SB","CS","BB","SO","TB","GDP","HBP","SH","SF","IBB",
    "BA","OBP","SLG","OPS"
]

for c in num_cols_bat:
    if c in bat_all.columns:
        bat_all[c] = pd.to_numeric(bat_all[c], errors="coerce")

# 저장
bat_all.to_csv(
    "kiwoom_batting_2021_2025_merged.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ 타자 병합 파일 저장 완료:", bat_all.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'BR_KIWOOM_batting_2021.csv'

In [5]:
pitchers = []

for year in YEARS:
    df = pd.read_csv(f"BR_KIWOOM_pitching_{year}.csv")
    df["season"] = year
    pitchers.append(df)

pit_all = pd.concat(pitchers, ignore_index=True)

# 합계 행 제거
if "Rk" in pit_all.columns:
    pit_all = pit_all[pit_all["Rk"].notna()].copy()

# 이름 정리
pit_all["Name"] = pit_all["Name"].astype(str).str.replace("*", "", regex=False)

# 숫자형 컬럼 정리
num_cols_pit = [
    "Age","G","GS","IP","H","R","ER","HR","BB","SO",
    "ERA","FIP","WHIP"
]

for c in num_cols_pit:
    if c in pit_all.columns:
        pit_all[c] = pd.to_numeric(pit_all[c], errors="coerce")

# 저장
pit_all.to_csv(
    "kiwoom_pitching_2021_2025_merged.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ 투수 병합 파일 저장 완료:", pit_all.shape)


✅ 투수 병합 파일 저장 완료: (146, 34)


In [3]:
import pandas as pd
from pathlib import Path

bat_path = Path("KBOData/Batting_KBO")
bat_dfs = []

for file in bat_path.glob("*.csv"):
    season = int(file.stem.split("_")[-1])
    df = pd.read_csv(file)
    df["season"] = season
    bat_dfs.append(df)

bat_kbo_all = pd.concat(bat_dfs, ignore_index=True)

# 저장
bat_kbo_all.to_csv(
    "KBO_batting_2021_2025_merged.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ KBO Batting 병합 완료:", bat_kbo_all.shape)



✅ KBO Batting 병합 완료: (55, 29)


In [4]:
pit_path = Path("KBOData/Pitching_KBO")
pit_dfs = []

for file in pit_path.glob("*.csv"):
    season = int(file.stem.split("_")[-1])
    df = pd.read_csv(file)
    df["season"] = season
    pit_dfs.append(df)

pit_kbo_all = pd.concat(pit_dfs, ignore_index=True)

pit_kbo_all.to_csv(
    "KBO_pitching_2021_2025_merged.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ KBO Pitching 병합 완료:", pit_kbo_all.shape)


✅ KBO Pitching 병합 완료: (55, 35)


In [5]:
fld_path = Path("KBOData/Fielding_KBO")
fld_dfs = []

for file in fld_path.glob("*.csv"):
    season = int(file.stem.split("_")[-1])
    df = pd.read_csv(file)
    df["season"] = season
    fld_dfs.append(df)

fielding_kbo_all = pd.concat(fld_dfs, ignore_index=True)

fielding_kbo_all.to_csv(
    "KBO_fielding_2021_2025_merged.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ KBO Fielding 병합 완료:", fielding_kbo_all.shape)


✅ KBO Fielding 병합 완료: (55, 19)


# 전처리 

## 1단계: 컬럼 정리 

### - 2024년 이전 팀 이름 칼럼 : finals, 24년 이후 팀 이름 칼럼 : Tm 
### - 비어져있는 컬럼 삭제  

In [ ]:
import pandas as pd

file_path = "KBO_batting_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# Finals → Tm 통일
if "Tm" not in df.columns and "Finals" in df.columns:
    df = df.rename(columns={"Finals": "Tm"})
    print("✅ Finals → Tm 변경 완료")
elif "Tm" in df.columns and "Finals" in df.columns:
    df["Tm"] = df["Tm"].fillna(df["Finals"])
    df = df.drop(columns=["Finals"])
    print("✅ Tm 유지 + Finals 병합 후 제거")
else:
    print("⏩ 변경 필요 없음")

# 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")


In [8]:
import pandas as pd

file_path = "KBO_batting_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# 1️⃣ Tm 컬럼 맨 앞으로 이동
if "Tm" not in df.columns:
    raise ValueError("❌ Tm 컬럼이 존재하지 않습니다.")

cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index("Tm")))
df = df[cols]

# 2️⃣ 값이 전부 비어있는 컬럼 삭제
df = df.dropna(axis=1, how="all")

# 3️⃣ 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✅ 처리 완료")
print("현재 컬럼 수:", df.shape[1])
print("컬럼 목록:", df.columns.tolist())


✅ 처리 완료
현재 컬럼 수: 27
컬럼 목록: ['Tm', 'BatAge', 'R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'season']


In [9]:
import pandas as pd

file_path = "KBO_pitching_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# Finals → Tm 통일
if "Tm" not in df.columns and "Finals" in df.columns:
    df = df.rename(columns={"Finals": "Tm"})
    print("✅ Finals → Tm 변경 완료")
elif "Tm" in df.columns and "Finals" in df.columns:
    df["Tm"] = df["Tm"].fillna(df["Finals"])
    df = df.drop(columns=["Finals"])
    print("✅ Tm 유지 + Finals 병합 후 제거")
else:
    print("⏩ 변경 필요 없음")

# 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")

✅ Tm 유지 + Finals 병합 후 제거


In [10]:
import pandas as pd

file_path = "KBO_pitching_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# 1️⃣ Tm 컬럼 맨 앞으로 이동
if "Tm" not in df.columns:
    raise ValueError("❌ Tm 컬럼이 존재하지 않습니다.")

cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index("Tm")))
df = df[cols]

# 2️⃣ 값이 전부 비어있는 컬럼 삭제
df = df.dropna(axis=1, how="all")

# 3️⃣ 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✅ 처리 완료")
print("현재 컬럼 수:", df.shape[1])
print("컬럼 목록:", df.columns.tolist())

✅ 처리 완료
현재 컬럼 수: 33
컬럼 목록: ['Tm', 'PAge', 'R/G', 'W', 'L', 'W-L%', 'ERA', 'RA9', 'G', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'season']


In [11]:
import pandas as pd

file_path = "KBO_fielding_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# Finals → Tm 통일
if "Tm" not in df.columns and "Finals" in df.columns:
    df = df.rename(columns={"Finals": "Tm"})
    print("✅ Finals → Tm 변경 완료")
elif "Tm" in df.columns and "Finals" in df.columns:
    df["Tm"] = df["Tm"].fillna(df["Finals"])
    df = df.drop(columns=["Finals"])
    print("✅ Tm 유지 + Finals 병합 후 제거")
else:
    print("⏩ 변경 필요 없음")

# 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")

✅ Tm 유지 + Finals 병합 후 제거


In [12]:
import pandas as pd

file_path = "KBO_fielding_2021_2025_merged.csv"

df = pd.read_csv(file_path)

# 1️⃣ Tm 컬럼 맨 앞으로 이동
if "Tm" not in df.columns:
    raise ValueError("❌ Tm 컬럼이 존재하지 않습니다.")

cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index("Tm")))
df = df[cols]

# 2️⃣ 값이 전부 비어있는 컬럼 삭제
df = df.dropna(axis=1, how="all")

# 3️⃣ 덮어쓰기 저장
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("✅ 처리 완료")
print("현재 컬럼 수:", df.shape[1])
print("컬럼 목록:", df.columns.tolist())

✅ 처리 완료
현재 컬럼 수: 13
컬럼 목록: ['Tm', 'G', 'PO', 'A', 'E', 'DP', 'Fld%', 'PB', 'SB', 'CS', 'CS%', 'Attendance', 'season']


# 2단계: 이상치 확인
    #의미없는 칼럼 삭제(ex.Attendance) 

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

FILES = [
    "KBO_batting_2021_2025_merged.csv",
    "KBO_pitching_2021_2025_merged.csv",
    "KBO_fielding_2021_2025_merged.csv",
    "kiwoom_batting_2021_2025_merged.csv",
    "kiwoom_pitching_2021_2025_merged.csv",
]

# 자주 등장하는 "비어있음" 표기들을 결측으로 통일
EMPTY_TOKENS = ["", " ", "  ", "-", "--", "—", "–", "NA", "N/A", "NaN", "nan", "null", "None"]

def basic_clean(df: pd.DataFrame) -> pd.DataFrame:
    # 문자열 양쪽 공백 제거
    df = df.copy()
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()

    # 빈 토큰 → NaN
    df = df.replace(EMPTY_TOKENS, np.nan)

    return df

def missing_report(df: pd.DataFrame, name: str) -> pd.DataFrame:
    miss_cnt = df.isna().sum()
    miss_pct = (df.isna().mean() * 100).round(2)
    out = pd.DataFrame({
        "file": name,
        "column": df.columns,
        "missing_count": miss_cnt.values,
        "missing_pct": miss_pct.values,
        "dtype": [str(t) for t in df.dtypes]
    })
    out = out.sort_values(["missing_count", "missing_pct"], ascending=False)
    return out

reports = []
cleaned = {}  # 이후 단계에서 재사용

for fp in FILES:
    df = pd.read_csv(fp)
    df = basic_clean(df)
    cleaned[fp] = df

    rep = missing_report(df, fp)
    reports.append(rep)

all_missing = pd.concat(reports, ignore_index=True)
all_missing.to_csv("missing_report_all_files.csv", index=False, encoding="utf-8-sig")

print("결측치 리포트 저장 완료: missing_report_all_files.csv")
print("결측치 있는 컬럼만 보기(상위 30개):")
display(all_missing[all_missing["missing_count"] > 0].head(30))


결측치 리포트 저장 완료: missing_report_all_files.csv
결측치 있는 컬럼만 보기(상위 30개):


,file,column,missing_count,missing_pct,dtype
60,KBO_fielding_2021_2025_merged.csv,Attendance,33,60.00,float64
73,kiwoom_batting_2021_2025_merged.csv,Notes,156,100.00,float64
102,kiwoom_pitching_2021_2025_merged.csv,Notes,146,100.00,float64
103,kiwoom_pitching_2021_2025_merged.csv,W-L%,35,23.97,float64
104,kiwoom_pitching_2021_2025_merged.csv,W,4,2.74,float64
105,kiwoom_pitching_2021_2025_merged.csv,L,4,2.74,float64
106,kiwoom_pitching_2021_2025_merged.csv,ERA,4,2.74,float64
107,kiwoom_pitching_2021_2025_merged.csv,G,4,2.74,float64
108,kiwoom_pitching_2021_2025_merged.csv,GS,4,2.74,float64
109,kiwoom_pitching_2021_2025_merged.csv,GF,4,2.74,float64


## Notes 칼럼 삭제 

In [14]:
import pandas as pd

targets = [
    "kiwoom_batting_2021_2025_merged.csv",
    "kiwoom_pitching_2021_2025_merged.csv"
]

for fp in targets:
    df = pd.read_csv(fp)

    if "Notes" in df.columns:
        df = df.drop(columns=["Notes"])
        df.to_csv(fp, index=False, encoding="utf-8-sig")
        print(f"{fp}: Notes 컬럼 삭제 후 덮어쓰기 완료")
    else:
        print(f"{fp}: Notes 컬럼 없음 (스킵)")


kiwoom_batting_2021_2025_merged.csv: Notes 컬럼 삭제 후 덮어쓰기 완료
kiwoom_pitching_2021_2025_merged.csv: Notes 컬럼 삭제 후 덮어쓰기 완료


## Attendance 칼럼 삭제 

In [15]:
import pandas as pd

fp = "KBO_fielding_2021_2025_merged.csv"
df = pd.read_csv(fp)

if "Attendance" in df.columns:
    df = df.drop(columns=["Attendance"])
    df.to_csv(fp, index=False, encoding="utf-8-sig")
    print(" Attendance 컬럼 삭제 및 덮어쓰기 완료")
else:
    print("Attendance 컬럼 없음 (스킵)")


 Attendance 컬럼 삭제 및 덮어쓰기 완료


In [16]:
import pandas as pd

fp = "kiwoom_pitching_2021_2025_merged.csv"
df = pd.read_csv(fp)

# 결측치가 하나라도 있는 행 확인
suspect_rows = df[df.isna().any(axis=1)]

suspect_rows[["Name", "season"]].head(10)


,Name,season
3,Young Gun Cho,2021
7,Chan Heon Jeong,2021
10,Dong Uk Kim,2021
11,In-beom Kim,2021
14,Joon Hyoung Kim,2021
19,Jong Min Lee,2021
21,Gyu Bin Lim,2021
22,Sung Hyun Moon,2021
24,Gwan Jin Park,2021
26,Seung Joo Park,2021


In [17]:
import pandas as pd

fp = "kiwoom_pitching_2021_2025_merged.csv"
df = pd.read_csv(fp)

# 투수 핵심 지표
core_cols = ["IP", "ERA", "WHIP", "SO", "BB", "H", "R", "ER"]
core_cols = [c for c in core_cols if c in df.columns]

before = len(df)

# 핵심 지표가 전부 NaN인 행 제거 (출장 없는 시즌)
df = df.dropna(subset=core_cols, how="all")

after = len(df)

df.to_csv(fp, index=False, encoding="utf-8-sig")

print(f"✅ 출장 없는 시즌 행 제거: {before - after}행 삭제")


✅ 출장 없는 시즌 행 제거: 4행 삭제


In [19]:
import pandas as pd

fp = "kiwoom_batting_2021_2025_merged.csv"
df = pd.read_csv(fp)

# 결측치가 하나라도 있는 행 확인
suspect_rows = df[df.isna().any(axis=1)]

suspect_rows[["Name", "season"]].head(10)

,Name,season


### 결측치 처리 

In [20]:
import pandas as pd
import numpy as np

fp = "kiwoom_pitching_2021_2025_merged.csv"
df = pd.read_csv(fp)

if "W-L%" in df.columns and {"W", "L"}.issubset(df.columns):
    df["W-L%"] = np.where(
        (df["W"] + df["L"]) > 0,
        df["W"] / (df["W"] + df["L"]),
        np.nan
    )

df.to_csv(fp, index=False, encoding="utf-8-sig")
print(" kiwoom_pitching W-L% 재계산 완료")


 kiwoom_pitching W-L% 재계산 완료


In [22]:
zero_cols_pitching = [
    "IBB", "HBP", "BK", "WP", "CG", "SHO", "SV", "GF"
]

df = pd.read_csv("kiwoom_pitching_2021_2025_merged.csv")

for c in zero_cols_pitching:
    if c in df.columns:
        df[c] = df[c].fillna(0)

df.to_csv("kiwoom_pitching_2021_2025_merged.csv", index=False, encoding="utf-8-sig")
print(" kiwoom_pitching 0-fill 완료")


 kiwoom_pitching 0-fill 완료


zero_cols_batting = [
    "SB", "CS", "SH", "SF", "IBB", "HBP", "GDP"
]

fp = "kiwoom_batting_2021_2025_merged.csv"
df = pd.read_csv(fp)

for c in zero_cols_batting:
    if c in df.columns:
        df[c] = df[c].fillna(0)

df.to_csv(fp, index=False, encoding="utf-8-sig")
print(" kiwoom_batting 0-fill 완료")


In [ ]:
### 데이터 가공 결과 확인 

In [24]:
files = [
    "kiwoom_batting_2021_2025_merged.csv",
    "kiwoom_pitching_2021_2025_merged.csv"
]

for f in files:
    df = pd.read_csv(f)
    print("\n", f)
    print(df.isna().sum().sort_values(ascending=False).head(10))



 kiwoom_batting_2021_2025_merged.csv
Rk      0
Name    0
Age     0
G       0
PA      0
AB      0
R       0
H       0
2B      0
3B      0
dtype: int64

 kiwoom_pitching_2021_2025_merged.csv
W-L%    31
Rk       0
Name     0
W        0
Age      0
L        0
ERA      0
G        0
GS       0
GF       0
dtype: int64
